## Create ENV

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

## Idea

Create a SQL script that can recreate the DB from scratch

## Cons vs Pros

### Advantages

- System independent
- Version independent
- Can dump a single DB
- Potentially pretty small - don't include index data, just instructions for creation

### Disadvantages

- Slow to produce
- Slow to restore
- Dumps a single DB at a time so consistency between DB's can be lost 

### Main Use Cases

- Migrate between versions and different architecture systems
- Backup individual parts of the cluster, specific DBs

## Demo

### Single DB

In [ ]:
# Create a db with some data
psql << EOM
    DROP DATABASE IF EXISTS example;
    CREATE DATABASE example;
    \c example
    DROP TABLE IF EXISTS foo;

    SELECT a, b
    INTO foo
    FROM generate_series(1,10000) a, generate_series(1,100) b;
EOM

In [ ]:
# Perform a sql dump
pg_dump example > example_dump_file.txt
head -n 50 example_dump_file.txt

In [ ]:
# Remove DB and restore
dropdb example
createdb example
psql example < example_dump_file.txt
psql -d example -c "SELECT * FROM foo LIMIT 10"
# Cleanup
rm example_dump_file.txt

### Entire Cluster

In [ ]:
# Create new directory
DIR=/some/new/cluster
rm -rf $DIR
mkdir -p $DIR
chown -R postgres $DIR
su postgres -c "initdb -D $DIR"

In [ ]:
# Dump
pg_dumpall -f $DIR/dump_all_example.txt
head -n 100 $DIR/dump_all_example.txt

In [ ]:
# Start new cluster
su postgres -c "pg_ctl -D $DIR -o '-p 5433' start"

In [ ]:
# Restore
psql -p 5433 -f $DIR/dump_all_example.txt postgres

In [ ]:
# Cleanup
su postgres -c "pg_ctl -D $DIR stop"
rm -r /some